In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_data.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [7]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [9]:
X_train.shape , X_test.shape

((80, 5), (20, 5))

# Without using Column Transformer

In [28]:
#simple imputer to fever column

si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [25]:
#one hot encoding to gender and city

ohe = OneHotEncoder(drop= 'first' , sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender' , 'city']])
X_test_gender_city = ohe.transform(X_test[['gender' , 'city']])

X_train_gender_city.shape

(80, 4)

In [26]:
#ordinal encoding to cough column

oe = OrdinalEncoder(categories=[['Mild' , 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [22]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [29]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# Using column transformer

In [31]:
from sklearn.compose import ColumnTransformer

In [32]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])],remainder='passthrough')

In [33]:
transformer.fit_transform(X_train).shape

(80, 7)

In [34]:
transformer.transform(X_test).shape

(20, 7)